In [1]:
import json
import re
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import csv
from nltk.stem import SnowballStemmer

In [2]:
def data_preporation(filename):
    labels = []
    titles = []
    texts = []
    
    with open('../input/text-classification-lenta/' + filename) as f:
        templates = json.load(f)

    for section, commands in templates.items():
        labels.append(section)
        texts.append(commands['texts'])
        titles.append(commands['titles'])
   
    for i in range(len(titles)):
        for j in range(len(titles[i])):
            texts[i][j] += ' ' + titles[i][j]
            
    labels_dict = {'Экономика': 0, 'Спорт': 1, 'Культура': 2, 'Наука и техника': 3}

    tmp_labels = [labels_dict[label] for label in labels]

    correct_labels = []

    for i in range(len(texts)):
        for j in range(len(texts[i])):
            correct_labels.append(tmp_labels[i])

    preproc_texts = []
    
    for text in texts:
        for i in text:
            s = i.lower()
            s = re.sub("[^а-яА-Яa-zA-Z0-9]", " ", s)
            s = re.sub("\s+", " ", s)
            s = s.strip()
            s = SnowballStemmer("russian").stem(s)
            preproc_texts += [s]

    return preproc_texts, correct_labels

def label_reverse(preds):
    labels_dict_reverse = {0: 'Экономика', 1: 'Спорт', 2: 'Культура', 3: 'Наукаитехника'}
    answ = [labels_dict_reverse[label] for label in preds]
    return answ

def learning_func(filename):
    preproc_texts, correct_labels = data_preporation(filename)
    
    tfidf = TfidfVectorizer()
    tfidf = tfidf.fit(preproc_texts)
    tf_idf_train_data = tfidf.transform(preproc_texts)

    svm_tfid_cls = LinearSVC(loss="squared_hinge", C=1.0, dual=False)
    svm_tfid_cls.fit(tf_idf_train_data, correct_labels)

    tf_idf_mtest_data = tfidf.transform(preproc_mtest_data)
    mtest_preds = svm_tfid_cls.predict(tf_idf_mtest_data)
    
    return label_reverse(mtest_preds)

In [3]:
mtest_data = []

with open('../input/text-classification-lenta/test.csv', newline = '') as file:
    reader = csv.reader(file)
    next(reader)
    for row in reader:
        mtest_data.append(row[1] + ' ' + row[0])

preproc_mtest_data = []

for text in mtest_data:
    s = text.lower()
    s = re.sub("[^а-яА-Яa-zA-Z0-9]", " ", s)
    s = re.sub("\s+", " ", s)
    s = s.strip()
    s = SnowballStemmer("russian").stem(s)
    preproc_mtest_data += [s]

In [4]:
answ = []
files = ['train_5.json', 'train_10.json', 'train_20.json',
         'train_50.json', 'train_100.json', 'train_1000.json', 'train_5000.json', 'train_10000.json',]

answ += learning_func(files[0])
answ += learning_func(files[1])
answ += learning_func(files[2])
answ += learning_func(files[3])
answ += learning_func(files[4])
len(answ)

2085

In [5]:
f = open('./submission_template.txt', 'w')
f.write('Id,Category' + '\n')
for i in range(len(answ)):
    f.write(str(i) + ',' + answ[i] + '\n')
f.close()